In [1]:
#Instalación de Paquetes para interactuar con el API de Reddit
#!pip install praw
#!pip install psaw


In [12]:
import praw
from psaw import PushshiftAPI

In [1]:
import datetime 
import pandas as pd


Keys del API

In [14]:
reddit = praw.Reddit(client_id='r0CFZHkOYlRvtw', client_secret='d03kgRzEp9HgNZ23voMPO6f-5EzjvA', user_agent='Sentiment analysis research')
api = PushshiftAPI(reddit)

Data del API a DF

In [15]:
def SubToDF (list_sub):
    posts=[]
    for post in list_sub:
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
    def get_date(created):
            return dt.datetime.fromtimestamp(created)
    _timestamp = posts["created"].apply(get_date)
    posts = posts.assign(created = _timestamp)
    return posts


In [2]:
def Get_Reddit_Data(Subreddit,period_1,period_2):
    query = list(api.search_submissions(subreddit=Subreddit, after=period_1, before=period_2, limit=None))
    return(SubToDF(query))

Obteniendo data desde 6/20 hasta 1/21

In [29]:
periodo_ini=int(datetime.datetime(2020, 6, 1).timestamp())
periodo_fin=int(datetime.datetime(2021, 1, 30).timestamp())

DF_WSP = Get_Reddit_Data("Whatsapp",periodo_ini,periodo_fin)
DF_TLGRM= Get_Reddit_Data("Telegram",periodo_ini,periodo_fin)
DF_SIGNAL=Get_Reddit_Data("Signal",periodo_ini,periodo_fin)

In [ ]:
DF_WSP.head()

In [ ]:
DF_TLGRM.head()

In [ ]:
DF_SIGNAL.head()

Obteniendo comentarios de cada post

In [30]:
def get_comment(id_rdt:str):
    text=[]
    submission = reddit.submission(id=id_rdt)
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]  # Seed with top-level
    while comment_queue:
        comment = comment_queue.pop(0)
        if comment.body != None : 
            text.append(comment.body)
        if comment_queue.extend(comment.replies) != None :
            text.append(comment_queue.extend(comment.replies) )
    return ' '.join(text)

In [31]:

DF_WSP['comments'] = DF_WSP.id.apply(get_comment)
#ABC['comments'] = ABC.id.apply(get_comment)


In [32]:
DF_SIGNAL['comments'] = DF_SIGNAL.id.apply(get_comment)


In [33]:
DF_TLGRM['comments'] = DF_TLGRM.id.apply(get_comment)

Guardando DF

In [34]:
DF_WSP.to_csv('Whatsapp.csv', sep=';',header=True, index=False)
DF_TLGRM.to_csv('Telegram.csv', sep=';',header=True, index=False)
DF_SIGNAL.to_csv('Signal.csv', sep=';',header=True, index=False)

In [35]:
df=pd.read_csv('Whatsapp.csv', sep=';')

In [27]:
df.sample(10)

,title,score,id,subreddit,url,num_comments,body,created,comments
407,Conversation history on new devices,3,kz0w82,signal,https://www.reddit.com/r/signal/comments/kz0w8...,4,"Hi all,\n\nJust tried installing Signal on des...",2021-01-17 09:07:59,"With Android & iOS, there's ways to transfer y..."
1114,So just downloaded Signal and it is not showin...,1,kw117n,signal,https://i.redd.it/46f9lz2oxya61.jpg,5,NaN,2021-01-13 00:21:31,Maybe it’s in your App-Library? I just opened ...
42,Signal is cool and I wish it was the universal...,0,l24sv2,signal,https://www.reddit.com/r/signal/comments/l24sv...,8,"Also, it's important to remember that most use...",2021-01-21 21:48:20,I had doubts too but all the new people I knew...
1725,Signal don't send message to register.,12,kszynp,signal,https://www.reddit.com/r/signal/comments/kszyn...,10,I want to register to signal but the app don't...,2021-01-08 13:34:13,"Good morning, folks. We did just resolve an is..."
1416,"error retrieving contacts, check your network ...",1,kulvit,signal,https://www.reddit.com/r/signal/comments/kulvi...,1,[removed],2021-01-10 23:11:30,This is clearly a server-side issue. Signal is...
322,Should there be a sticky 'new user' thread?,1,kzffue,signal,https://www.reddit.com/r/signal/comments/kzffu...,1,[removed],2021-01-18 00:28:05,"Thank you for the suggestion! In the future, p..."
104,Help your friends switch over with this explai...,2,l1a618,signal,https://www.reddit.com/r/signal/comments/l1a61...,1,[removed],2021-01-20 17:25:41,"Thank you for your submission! Unfortunately, ..."
1683,Language,0,ktaafc,signal,https://www.reddit.com/r/signal/comments/ktaaf...,1,[deleted],2021-01-08 22:55:11,See [Signal Support - Language Options](https:...
607,Amazon will donate to Signal for you via this ...,1,ky3i7q,signal,https://smile.amazon.com/ch/82-4506840,2,NaN,2021-01-15 23:49:13,"Thank you for your submission! Unfortunately, ..."
345,Compare!,124,kzb655,signal,https://www.reddit.com/gallery/kzb655,26,NaN,2021-01-17 20:55:11,Facebook wants all the data.\n\nPeople really ...


In [36]:
df[df.body.notna()]

,title,score,id,subreddit,url,num_comments,body,created,comments
0,"German Stickers ""1Typ"" from Fehlgeburt",1,l6feyk,whatsapp,https://www.reddit.com/r/whatsapp/comments/l6f...,0,[https://play.google.com/store/apps/details?id...,2021-01-28 00:53:51,NaN
1,What did WhatsApp update this time?,1,l6egin,whatsapp,https://www.reddit.com/r/whatsapp/comments/l6e...,1,I heard so many people saying that WhatsApp up...,2021-01-28 00:09:03,No. WhatsApp remains end to end encrypted and ...
6,QR scanner not working,1,l6al8f,whatsapp,https://www.reddit.com/r/whatsapp/comments/l6a...,0,I try scan the code and the same message keeps...,2021-01-27 21:04:45,NaN
7,https://chat.whatsapp.com/D8bpCYCZbquKtiiIJgmFi9,1,l693bt,whatsapp,https://www.reddit.com/r/whatsapp/comments/l69...,0,[removed],2021-01-27 20:04:51,NaN
8,"Sorry, this media file appears to be missing.",1,l5hd0z,whatsapp,https://www.reddit.com/r/whatsapp/comments/l5h...,2,I am currently trying to download a pdf to vie...,2021-01-26 19:21:41,The files are removed from the WA servers afte...
...,...,...,...,...,...,...,...,...,...
6074,Is there another way to connect to WA web?,4,gulmh7,whatsapp,https://www.reddit.com/r/whatsapp/comments/gul...,3,I'm having trouble with my camera and can't sc...,2020-06-01 17:29:51,"No I dont know if it is possible, but i have i..."
6078,whatsapp pc,1,guk9it,whatsapp,https://www.reddit.com/r/whatsapp/comments/guk...,1,"hey, does anyone know if u can set urself to s...",2020-06-01 16:03:15,Minimize the window if you're not using it oth...
6079,WHATSAPP लेटेस्ट अपडेट 2020,0,guhd1k,whatsapp,https://www.reddit.com/r/whatsapp/comments/guh...,0,[removed],2020-06-01 12:18:22,NaN
6080,Got a weird whatsapp video call,5,gug0e8,whatsapp,https://www.reddit.com/r/whatsapp/comments/gug...,2,So I repeatedly got video calls on WhatsApp fr...,2020-06-01 10:23:01,That sounds like someone else had access to hi...
